Import Libraries

In [1]:
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
from scipy import interpolate

# Import pandas
import pandas as pd

# Import numpy
import numpy as np
from numpy import * 
from numpy.linalg import multi_dot
import numpy_financial as npf

# Plot settings
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = 16, 8

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# IBT or Pension or Retirement
PLAN = 'IBT'

UPS_Contr_Pctg = 0

In [2]:
df_CF = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "PBO Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF

,Time,Retirement,Pension,IBT
Year,,,,
2021-12-31,0.5,1.204791e+09,2.626341e+08,4.239987e+08
2022-12-31,1.5,1.261684e+09,2.935988e+08,4.570359e+08
2023-12-31,2.5,1.322496e+09,3.253961e+08,4.883641e+08
2024-12-31,3.5,1.382473e+09,3.576365e+08,5.168109e+08
2025-12-31,4.5,1.441835e+09,3.898680e+08,5.408954e+08
...,...,...,...,...
2096-12-31,75.5,1.769334e+05,3.948190e+06,1.620103e+06
2097-12-31,76.5,1.503230e+05,2.959047e+06,1.183430e+06
2098-12-31,77.5,1.300395e+05,2.180928e+06,8.499443e+05


In [3]:
df_CF = df_CF.divide(12)
df_CF

,Time,Retirement,Pension,IBT
Year,,,,
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-12-31,0.125000,1.051403e+08,2.446657e+07,3.808632e+07
2023-12-31,0.208333,1.102080e+08,2.711634e+07,4.069701e+07
2024-12-31,0.291667,1.152061e+08,2.980304e+07,4.306757e+07
2025-12-31,0.375000,1.201529e+08,3.248900e+07,4.507462e+07
...,...,...,...,...
2096-12-31,6.291667,1.474445e+04,3.290158e+05,1.350085e+05
2097-12-31,6.375000,1.252692e+04,2.465872e+05,9.861913e+04
2098-12-31,6.458333,1.083662e+04,1.817440e+05,7.082869e+04


In [4]:
df=df_CF.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_CF = df
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-02-28,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-03-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-04-30,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04


In [5]:
df_CF_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "CF", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF_add

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,19389650.5,31075638.75
2021-11-30,0.166667,9.387313e+07,19389650.5,31075638.75
2021-12-31,0.250000,9.387313e+07,19389650.5,31075638.75


In [6]:
df_CF  = df_CF_add.append(df_CF)
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04


In [7]:
from itertools import count, takewhile
def frange(start, stop, step):
        return takewhile(lambda x: x< stop, count(start, step))
t = list(frange((1/12), 79.41666666666, (1/12)))

In [8]:
df_t=pd.DataFrame(t)
df_t

,0
0,0.083333
1,0.166667
2,0.250000
3,0.333333
4,0.416667
...,...
947,79.000000
948,79.083333
949,79.166667
950,79.250000


In [9]:
df_CF['Time'] = df_t.values
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.333333,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.416667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,79.000000,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,79.083333,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,79.166667,9.606686e+03,1.315861e+05,4.997286e+04


In [10]:
#IBT = df_CF[PLAN]
#RP = df_CF[PLAN]
CF = np.array(df_CF[PLAN])
Time_CF = df_CF.index
DF = np.array(df_CF['Time'])
CF

array([3.10756388e+07, 3.10756388e+07, 3.10756388e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.50746205e+07,
       4.50746205e+07, 4.50746205e+07, 4.50746205e+07, 4.50746205e+07,
      

In [11]:
from PyCurve.curve import Curve

In [12]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 145)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,2010-09-30 00:00:00,2010-08-31 00:00:00,2010-07-31 00:00:00,2010-06-30 00:00:00,2010-05-31 00:00:00,2010-04-30 00:00:00,2010-03-31 00:00:00,2010-02-28 00:00:00,2010-01-31 00:00:00,2009-12-31 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,0.896931,0.980827,1.039656,1.336495,1.356106,0.796529,0.824371,0.959171,0.854822,0.997499
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,1.012479,1.093406,1.156291,1.494602,1.548488,1.069795,1.179135,1.183362,1.099006,1.386799
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,1.065754,1.180900,1.244818,1.590332,1.686592,1.323298,1.459969,1.412837,1.358586,1.664896
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,1.132115,1.250318,1.346470,1.693166,1.853045,1.533786,1.684716,1.612329,1.567095,1.892898
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,1.289564,1.400126,1.517540,1.883170,2.099087,1.825434,1.982464,1.898602,1.860517,2.202034
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,1.480375,1.593382,1.723042,2.141054,2.396014,2.163991,2.328185,2.229761,2.204905,2.555180
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,1.695568,1.812045,1.960053,2.417479,2.679186,2.485899,2.676519,2.544241,2.541772,2.898791
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,1.924112,2.035995,2.214525,2.671984,2.922165,2.770801,2.999055,2.823711,2.849782,3.213513
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,2.149881,2.249067,2.462229,2.882171,3.138009,3.038841,3.281442,3.099499,3.139972,3.515488
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,2.374888,2.459324,2.706179,3.066176,3.344929,3.302149,3.534706,3.380740,3.421418,3.802621


In [13]:
raw_liab_ibt_dict={}
tr = Raw_FTSE_YC['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict[col] = Beg_Rate + y + End_Rate
raw_liab_ibt_dict[col]
#curve = Curve(t,y)
#print("curve.get_rate:{},curve.get_time:{}".format(curve.get_rate,curve.get_time))
#print(curve.get_rate)
#print(curve.get_time)

[0.9974989024527092,
 0.9974989024527092,
 0.9974989024527092,
 0.9974989024527092,
 0.9974989024527092,
 0.9974989024527092,
 1.062382201353033,
 1.127265500253357,
 1.1921487991536805,
 1.2570320980540044,
 1.3219153969543282,
 1.386798695854652,
 1.4331483013106938,
 1.4794979067667358,
 1.5258475122227777,
 1.5721971176788196,
 1.6185467231348616,
 1.6648963285909035,
 1.7028965521970638,
 1.7408967758032243,
 1.7788969994093848,
 1.8168972230155453,
 1.8548974466217059,
 1.8928976702278664,
 1.9444204380138232,
 1.9959432057997806,
 2.0474659735857377,
 2.098988741371695,
 2.1505115091576523,
 2.2020342769436096,
 2.2608919641507463,
 2.319749651357883,
 2.37860733856502,
 2.437465025772157,
 2.4963227129792935,
 2.5551804001864302,
 2.612448907353061,
 2.669717414519691,
 2.726985921686321,
 2.784254428852951,
 2.8415229360195813,
 2.898791443186212,
 2.9512449994413394,
 3.003698555696467,
 3.0561521119515946,
 3.1086056682067222,
 3.1610592244618503,
 3.213512780716978,
 3.2638

In [14]:
int_df = pd.DataFrame(raw_liab_ibt_dict)
int_df = int_df.drop(['Date'], axis=1)
int_df = int_df.iloc[:, ::-1]
int_df

,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
0,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
1,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
2,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
3,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
4,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
948,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
949,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
950,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599


In [15]:
liab_plan_dict={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df[col][j]/100)**DF[j]))
    liab_plan_dict[col] = temp_pv
len(liab_plan_dict)

142

In [16]:
pv_df = pd.DataFrame(liab_plan_dict, index = ['Present Values']).transpose()
col = pv_df.columns
np.negative(pv_df['Present Values'][0])

-9441977545.491364

In [17]:
Plan_Return_YC = np.zeros(len(liab_plan_dict)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC[i] = ((pv_df['Present Values'][i+1])/pv_df['Present Values'][i])-1
Plan_Return_YC

array([ 9.42865717e-03, -7.21562554e-03, -8.87648229e-03,  3.67743780e-02,
       -4.95219251e-03,  5.12721060e-02,  1.06388756e-02,  5.75074062e-02,
       -1.38859807e-02, -3.79158140e-02, -6.56062121e-03, -1.63883857e-02,
       -3.11037341e-02,  1.69336215e-02, -1.38167844e-02,  2.30058404e-02,
        1.85198215e-02, -2.78849551e-02,  4.63135011e-02,  2.31859042e-02,
        7.33892885e-02, -4.45204518e-04,  6.06054040e-04,  4.55745101e-02,
       -4.07416611e-03,  1.94650863e-03, -3.51593719e-02,  2.13459578e-02,
        3.16994260e-02,  3.45324164e-02,  6.68892757e-02, -1.48678257e-02,
       -1.70845231e-02,  1.86894075e-02, -1.27808231e-02, -2.24026719e-02,
       -4.00684167e-02,  7.69536834e-03, -9.30247813e-03,  4.08419403e-02,
       -6.73129263e-02, -4.76998550e-02, -1.49139088e-03, -7.42057291e-03,
       -7.00084844e-03,  1.70391373e-02, -1.72789513e-02, -5.30234015e-03,
        4.17969220e-02,  1.43274467e-02,  5.67373578e-03,  1.67103052e-02,
        1.90175386e-02, -

In [18]:
Plan_Returns_YC = pd.DataFrame(Plan_Return_YC)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC.to_excel(writer, sheet_name='returns')
pv_df.to_excel(writer, sheet_name='pv')
writer.save()

In [19]:
df_SC = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "Service Cost Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC

,Retirement,Pension,IBT
Year,,,
2021-12-31,1511573.39,1.515507e+05,8.562248e+05
2022-12-31,4341798.12,6.378839e+05,3.265973e+06
2023-12-31,6325185.11,1.364457e+06,6.377871e+06
2024-12-31,8362114.74,2.184367e+06,9.520245e+06
2025-12-31,10514285.02,3.091714e+06,1.271774e+07
...,...,...,...
2096-12-31,2534.24,7.448368e+05,8.144005e+05
2097-12-31,1502.48,5.765053e+05,6.166164e+05
2098-12-31,873.21,4.384455e+05,4.589457e+05


In [20]:
df_SC = df_SC.divide(12)

In [21]:
df=df_SC.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_SC = df
df_SC

,Retirement,Pension,IBT
date,,,
2021-12-31,125964.449167,12629.223266,71352.069903
2022-01-31,125964.449167,12629.223266,71352.069903
2022-02-28,125964.449167,12629.223266,71352.069903
2022-03-31,125964.449167,12629.223266,71352.069903
2022-04-30,125964.449167,12629.223266,71352.069903
...,...,...,...
2100-08-31,41.463333,27272.299072,27955.270099
2100-09-30,41.463333,27272.299072,27955.270099
2100-10-31,41.463333,27272.299072,27955.270099


In [22]:
df_SC_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "SC", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC_add

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,184557.083333,4629.916667,70374.833333
2021-11-30,0.166667,184557.083333,4629.916667,70374.833333
2021-12-31,0.250000,184557.083333,4629.916667,70374.833333


In [23]:
df_SC  = df_SC_add.append(df_SC)
df_SC

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,184557.083333,4629.916667,70374.833333
2021-11-30,0.166667,184557.083333,4629.916667,70374.833333
2021-12-31,0.250000,184557.083333,4629.916667,70374.833333
2021-12-31,NaN,125964.449167,12629.223266,71352.069903
2022-01-31,NaN,125964.449167,12629.223266,71352.069903
...,...,...,...,...
2100-08-31,NaN,41.463333,27272.299072,27955.270099
2100-09-30,NaN,41.463333,27272.299072,27955.270099
2100-10-31,NaN,41.463333,27272.299072,27955.270099


In [24]:
col_SC = df_SC.columns
col_CF = df_CF.columns
df_CF_Tot = UPS_Contr_Pctg*df_SC[col_SC] + df_CF[col_CF]
df_CF_Tot

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,NaN,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,NaN,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,NaN,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,NaN,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,NaN,9.606686e+03,1.315861e+05,4.997286e+04


In [25]:
df_CF_Tot['Time'] = df_t.values
df_CF_Tot

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.333333,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.416667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,79.000000,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,79.083333,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,79.166667,9.606686e+03,1.315861e+05,4.997286e+04


In [26]:
CF_Tot = df_CF_Tot[PLAN]
Time_CF_Tot = df_CF_Tot.index
DF_Tot=df_CF_Tot['Time']

In [27]:
liab_plan_dict_tot={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF_Tot[j]/((1+int_df[col][j]/100)**DF_Tot[j]))
    liab_plan_dict_tot[col] = temp_pv
liab_plan_dict_tot


{datetime.datetime(2009, 12, 31, 0, 0): 9441977545.491364,
 datetime.datetime(2010, 1, 31, 0, 0): 9531002714.756456,
 datetime.datetime(2010, 2, 28, 0, 0): 9462230568.192614,
 datetime.datetime(2010, 3, 31, 0, 0): 9378239246.162477,
 datetime.datetime(2010, 4, 30, 0, 0): 9723118160.902142,
 datetime.datetime(2010, 5, 31, 0, 0): 9674967407.949291,
 datetime.datetime(2010, 6, 30, 0, 0): 10171023362.43103,
 datetime.datetime(2010, 7, 31, 0, 0): 10279231614.6664,
 datetime.datetime(2010, 8, 31, 0, 0): 10870363562.641134,
 datetime.datetime(2010, 9, 30, 0, 0): 10719417903.831335,
 datetime.datetime(2010, 10, 31, 0, 0): 10312982448.670992,
 datetime.datetime(2010, 11, 30, 0, 0): 10245322877.249722,
 datetime.datetime(2010, 12, 31, 0, 0): 10077418574.04642,
 datetime.datetime(2011, 1, 31, 0, 0): 9763973226.147701,
 datetime.datetime(2011, 2, 28, 0, 0): 9929312652.788202,
 datetime.datetime(2011, 3, 31, 0, 0): 9792121480.412828,
 datetime.datetime(2011, 4, 30, 0, 0): 10017397464.769346,
 datet

In [28]:
pv_df_tot = pd.DataFrame(liab_plan_dict_tot, index = ['Present Values']).transpose()

pv_df_tot

,Present Values
2009-12-31,9.441978e+09
2010-01-31,9.531003e+09
2010-02-28,9.462231e+09
2010-03-31,9.378239e+09
2010-04-30,9.723118e+09
...,...
2021-05-31,1.492173e+10
2021-06-30,1.556223e+10
2021-07-31,1.594887e+10
2021-08-31,1.580312e+10


In [29]:
Plan_Return_YC_Tot = np.zeros(len(liab_plan_dict_tot)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_tot)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Tot[i] = ((pv_df_tot['Present Values'][i+1])/pv_df_tot['Present Values'][i])-1
Plan_Return_YC_Tot

array([ 9.42865717e-03, -7.21562554e-03, -8.87648229e-03,  3.67743780e-02,
       -4.95219251e-03,  5.12721060e-02,  1.06388756e-02,  5.75074062e-02,
       -1.38859807e-02, -3.79158140e-02, -6.56062121e-03, -1.63883857e-02,
       -3.11037341e-02,  1.69336215e-02, -1.38167844e-02,  2.30058404e-02,
        1.85198215e-02, -2.78849551e-02,  4.63135011e-02,  2.31859042e-02,
        7.33892885e-02, -4.45204518e-04,  6.06054040e-04,  4.55745101e-02,
       -4.07416611e-03,  1.94650863e-03, -3.51593719e-02,  2.13459578e-02,
        3.16994260e-02,  3.45324164e-02,  6.68892757e-02, -1.48678257e-02,
       -1.70845231e-02,  1.86894075e-02, -1.27808231e-02, -2.24026719e-02,
       -4.00684167e-02,  7.69536834e-03, -9.30247813e-03,  4.08419403e-02,
       -6.73129263e-02, -4.76998550e-02, -1.49139088e-03, -7.42057291e-03,
       -7.00084844e-03,  1.70391373e-02, -1.72789513e-02, -5.30234015e-03,
        4.17969220e-02,  1.43274467e-02,  5.67373578e-03,  1.67103052e-02,
        1.90175386e-02, -

In [30]:
Plan_Returns_YC_Tot = pd.DataFrame(Plan_Return_YC_Tot)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability & SC Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Tot.to_excel(writer, sheet_name='returns')
pv_df_tot.to_excel(writer, sheet_name='pv')
writer.save()

In [31]:
Plan_DR = pd.read_excel("UPS Pension - Historical Liability Data - 9.30.21.xlsx",sheet_name = PLAN ,skiprows = [0,1,2,3],usecols=[1,2], na_values=[""],index_col=0)
#dataset.set_index('Year')
len(Plan_DR)


142

In [32]:
col_DR = Plan_DR.columns
#df_DR_M =  (1+Plan_DR)**(1/12)-1
df_DR_M = Plan_DR
IRR = df_DR_M['IRR']
IRR

Date
2009-12-31    0.059959
2010-01-31    0.058975
2010-02-28    0.059820
2010-03-31    0.060101
2010-04-30    0.057504
                ...   
2021-05-31    0.030762
2021-06-30    0.028554
2021-07-31    0.027088
2021-08-31    0.027607
2021-09-30    0.032400
Name: IRR, Length: 142, dtype: float64

In [33]:
Plan_PV_DR = np.zeros(len(df_DR_M))
for j in range (len(df_DR_M)):
    for i in range (len(Time_CF)):
        Plan_PV_DR[j] += (CF[i]/((1+IRR[j])**DF[i]))
Plan_PV_DR

array([9.29336565e+09, 9.42045483e+09, 9.31104495e+09, 9.27523915e+09,
       9.61590533e+09, 9.46050351e+09, 9.93155311e+09, 1.00908218e+10,
       1.06079164e+10, 1.05083114e+10, 1.01387245e+10, 1.00525102e+10,
       9.96091844e+09, 9.65217195e+09, 9.84778344e+09, 9.70762421e+09,
       9.94852305e+09, 1.01100236e+10, 9.83579907e+09, 1.01947065e+10,
       1.03539345e+10, 1.09062177e+10, 1.09699870e+10, 1.10204328e+10,
       1.15061017e+10, 1.14824835e+10, 1.15329407e+10, 1.12114178e+10,
       1.14341247e+10, 1.17581639e+10, 1.22349737e+10, 1.30105628e+10,
       1.28834870e+10, 1.27119687e+10, 1.29644103e+10, 1.28048406e+10,
       1.25407661e+10, 1.20454800e+10, 1.21517371e+10, 1.20283597e+10,
       1.25086580e+10, 1.16919306e+10, 1.11144060e+10, 1.11325889e+10,
       1.10718694e+10, 1.10035188e+10, 1.12151338e+10, 1.10278015e+10,
       1.09533609e+10, 1.14312761e+10, 1.15971597e+10, 1.16675468e+10,
       1.18574817e+10, 1.20951074e+10, 1.20765841e+10, 1.20680286e+10,
      

In [34]:
pv_df_dr = pd.DataFrame(Plan_PV_DR)

pv_df_dr

,0
0,9.293366e+09
1,9.420455e+09
2,9.311045e+09
3,9.275239e+09
4,9.615905e+09
...,...
137,1.471871e+10
138,1.532121e+10
139,1.574178e+10
140,1.559100e+10


In [35]:
pv_df_dr = pd.DataFrame(Plan_PV_DR)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability PV Using DR.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
pv_df_dr.to_excel(writer, sheet_name='Present Values')
writer.save()

In [36]:
Initial_CF = np.negative(pv_df_tot['Present Values'])
type(Initial_CF)


pandas.core.series.Series

In [37]:
CF

array([3.10756388e+07, 3.10756388e+07, 3.10756388e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.50746205e+07,
       4.50746205e+07, 4.50746205e+07, 4.50746205e+07, 4.50746205e+07,
      

In [38]:
DF


array([ 0.08333333,  0.16666667,  0.25      ,  0.33333333,  0.41666667,
        0.5       ,  0.58333333,  0.66666667,  0.75      ,  0.83333333,
        0.91666667,  1.        ,  1.08333333,  1.16666667,  1.25      ,
        1.33333333,  1.41666667,  1.5       ,  1.58333333,  1.66666667,
        1.75      ,  1.83333333,  1.91666667,  2.        ,  2.08333333,
        2.16666667,  2.25      ,  2.33333333,  2.41666667,  2.5       ,
        2.58333333,  2.66666667,  2.75      ,  2.83333333,  2.91666667,
        3.        ,  3.08333333,  3.16666667,  3.25      ,  3.33333333,
        3.41666667,  3.5       ,  3.58333333,  3.66666667,  3.75      ,
        3.83333333,  3.91666667,  4.        ,  4.08333333,  4.16666667,
        4.25      ,  4.33333333,  4.41666667,  4.5       ,  4.58333333,
        4.66666667,  4.75      ,  4.83333333,  4.91666667,  5.        ,
        5.08333333,  5.16666667,  5.25      ,  5.33333333,  5.41666667,
        5.5       ,  5.58333333,  5.66666667,  5.75      ,  5.83

In [39]:
def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

def irr(cfs, yrs, x0, **kwargs):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs), **kwargs))


NPV = np.zeros(len(df_DR_M))
for j in range (len(df_DR_M)):
    for i in range (len(Time_CF)):
        NPV[j] += npv(IRR[j], CF[i], DF[i])
NPV 

#IRR = np.zeros(len(df_DR_M))
#for j in range (len(df_DR_M)):
    #for i in range (len(Time_CF)):
        #IRR[j] += irr(CF[i], DF[i], .03)
#IRR 



array([9.29336565e+09, 9.42045483e+09, 9.31104495e+09, 9.27523915e+09,
       9.61590533e+09, 9.46050351e+09, 9.93155311e+09, 1.00908218e+10,
       1.06079164e+10, 1.05083114e+10, 1.01387245e+10, 1.00525102e+10,
       9.96091844e+09, 9.65217195e+09, 9.84778344e+09, 9.70762421e+09,
       9.94852305e+09, 1.01100236e+10, 9.83579907e+09, 1.01947065e+10,
       1.03539345e+10, 1.09062177e+10, 1.09699870e+10, 1.10204328e+10,
       1.15061017e+10, 1.14824835e+10, 1.15329407e+10, 1.12114178e+10,
       1.14341247e+10, 1.17581639e+10, 1.22349737e+10, 1.30105628e+10,
       1.28834870e+10, 1.27119687e+10, 1.29644103e+10, 1.28048406e+10,
       1.25407661e+10, 1.20454800e+10, 1.21517371e+10, 1.20283597e+10,
       1.25086580e+10, 1.16919306e+10, 1.11144060e+10, 1.11325889e+10,
       1.10718694e+10, 1.10035188e+10, 1.12151338e+10, 1.10278015e+10,
       1.09533609e+10, 1.14312761e+10, 1.15971597e+10, 1.16675468e+10,
       1.18574817e+10, 1.20951074e+10, 1.20765841e+10, 1.20680286e+10,
      